## test netcdf+

In [1]:
import openpathsampling as paths
from openpathsampling.netcdfplus import NetCDFPlus, ObjectStore
import numpy as np

### open file

try to create a new storage

In [2]:
st = NetCDFPlus('test_netcdfplus.nc', mode='w')

In [3]:
st.create_store('details', ObjectStore(paths.Details, has_name=False))

In [4]:
st.details._init()
st.details.set_caching(10)

In [5]:
st.create_variable('whatever', 'obj', dimensions=(10, ), chunksizes=(1, ), maskable=False)

<type 'netCDF4._netCDF4.Variable'>
int32 whatever(whatever_dim_0, pair)
    var_type: obj
unlimited dimensions: 
current shape = (10, 2)
filling on, default _FillValue of -2147483647 used

In [6]:
st.update_delegates()

In [7]:
st.save(paths.Details())[1:]

(0, 0)

In [12]:
st.variables['whatever'][1] = paths.Details()

TypeError: long() argument must be a string or a number, not 'Details'

In [8]:
st.variables['whatever'][0] = (0, 0)

In [8]:
st.sync()

In [9]:
print st.variables['details_json']
st.variables['whatever'][:]

<type 'netCDF4._netCDF4.Variable'>
vlen details_json(details)
    var_type: json
    long_str: details_json[#ix0:details] is A json serialized version of the object
vlen data type: <type 'str'>
unlimited dimensions: details
current shape = (0,)



masked_array(data =
 [[-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]
 [-- --]],
             mask =
 [[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]
 [ True  True]],
       fill_value = -2147483647)

### create variables types

In [5]:
print sorted(st.get_var_types())

['bool', 'float', 'index', 'int', 'json', 'lazyobj.details', 'lazyobj.stores', 'length', 'long', 'numpy.float32', 'numpy.float64', 'numpy.int16', 'numpy.int32', 'numpy.int64', 'numpy.int8', 'numpy.uint16', 'numpy.uint32', 'numpy.uint64', 'numpy.uint8', 'obj.details', 'obj.stores', 'store', 'str']


In [6]:
for var_type in st.get_var_types():
    st.create_variable(var_type, var_type, dimensions=(2, 2, 2))

In [7]:
st.update_delegates()

In [8]:
for var in sorted(st.variables):
    print var

bool
details_json
float
index
int
json
lazyobj.details
lazyobj.stores
length
long
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj.details
obj.stores
store
stores_json
stores_name
str


In [9]:
for var in sorted(st.vars):
    print var

bool
details_json
float
index
int
json
lazyobj.details
lazyobj.stores
length
long
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj.details
obj.stores
store
stores_json
stores_name
str


Bool

In [10]:
st.vars['bool'][:] = True

In [11]:
print st.vars['bool'][:]

[[[True, True], [True, True]], [[True, True], [True, True]]]


Float

In [12]:
st.vars['float'][1,1] = 1.0

In [13]:
print st.vars['float'][:]

[[[None, None], [None, None]], [[None, None], [1.0, 1.0]]]


Index

In [14]:
st.vars['index'][0,1,0] = 10
st.vars['index'][0,1,1] = -1

In [15]:
print st.vars['index'][0,1]

[10, None]


Int

In [16]:
st.vars['int'][0,1,0] = 10
st.vars['int'][0,1,1] = -1

In [17]:
print st.vars['int'][:]

[[[None, None], [10, -1]], [[None, None], [None, None]]]


JSON

A JSON serializable object. This can be normal very simple python objects, plus numpy arrays, and objects that implement `to_dict` and `from_dict`.

In [18]:
st.vars['json'][0,1,1] = {'Hallo': 2, 'Test': 3}

In [19]:
st.vars['json'][0,1,0]

In [8]:
st.vars['json'][0,1,0] = paths.Details(att = 2)

KeyError: 'json'

In [21]:
print st.variables['json'][0,1,:]

[u'{"_cls": "Details", "_dict": {}}' u'{"Test": 3, "Hallo": 2}']


All object types registered as being Storable by subclassing from `openpathsampling.base.StorableObject`. For this test we only register type `paths.Snapshot`

Numpy

In [22]:
st.vars['numpy.float32'][:] = np.ones((2,2,2)) * 3.0
st.vars['numpy.float32'][0] = np.ones((2,2)) * 7.0

In [23]:
print st.vars['numpy.float32'][:]

[[[ 7.  7.]
  [ 7.  7.]]

 [[ 3.  3.]
  [ 3.  3.]]]


Obj

In [24]:
st.vars['obj.details'][0,0,0] = paths.Details(cool='First')
st.vars['obj.details'][0,0,0] = paths.Details(cool='Second')
st.vars['obj.details'][0,0,0] = paths.Details(cool='Third')

In [25]:
print st.variables['obj.details'][:]
print st.variables['details_json'][:]

[[[2 --]
  [-- --]]

 [[-- --]
  [-- --]]]
[u'{"_cls": "Details", "_dict": {"cool": "First"}}'
 u'{"_cls": "Details", "_dict": {"cool": "Second"}}'
 u'{"_cls": "Details", "_dict": {"cool": "Third"}}']


In [26]:
print st.vars['obj.details'][0,0,0]
print type(st.vars['obj.details'][0,0,0])

cool = Third

<class 'openpathsampling.pathmover.Details'>


lazy

In [27]:
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='First')
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='Second')
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='Third')

In [28]:
proxy = st.vars['lazyobj.details'][0,0,0]
print 'Type:   ', type(proxy)
print 'Class:  ', proxy.__class__
print 'Content:', proxy.__subject__.__dict__
print 'Access: ', proxy.lazy

Type:    <class 'openpathsampling.netcdfplus.proxy.LoaderProxy'>
Class:   <class 'openpathsampling.pathmover.Details'>
Content: {'lazy': 'Third'}
Access:  Third


### load/save objects

Note that there are now 6 objects. 3 from `obj.details` and `lazyobj.details` each.

In [29]:
print st.details[:]

[<openpathsampling.pathmover.Details object at 0x11270e650>, <openpathsampling.pathmover.Details object at 0x112237b90>, <openpathsampling.pathmover.Details object at 0x112237c50>, <openpathsampling.pathmover.Details object at 0x11270e790>, <openpathsampling.pathmover.Details object at 0x11224bad0>, <openpathsampling.pathmover.Details object at 0x1060f6ed0>]


In [30]:
obj = paths.Details(name='BlaBla')
st.details.save(obj)
st.save(obj);

In [31]:
print len(st.details)

7


Note that we have only 7 objets since both save operations will only store one object.

Get the index of the obj in the storage

In [32]:
print st.details.variables['json'][st.idx(obj)]
print st.details.vars['json'][st.idx(obj)]
print st.details.vars['json'][st.idx(obj)] is obj
print st.details[st.idx(obj)] is obj

{"_cls": "Details", "_dict": {"name": "BlaBla"}}
name = BlaBla

False
True
